In [1]:
import os
from OPENAI_KEY import OPENAI_KEY
os.environ["OPENAI_API_KEY"] =  OPENAI_KEY

In [2]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files = ["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]), "\n")
# print((documents), "\n")

<class 'list'> 

41 

<class 'llama_index.core.schema.Document'> 


In [4]:
# Merge into a single large document for better RAG performance
from llama_index.core import Document

document = Document(text = "\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser

# create the sentence window node parser with default settings
# 
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

In [6]:
text = "hello. how are you? I am fine! "

# Split into individual sentences
nodes = node_parser.get_nodes_from_documents([Document(text=text)])

In [42]:
print(type(nodes))
print([x.text for x in nodes])
print(nodes[0].metadata["window"]) # print the nodes around the sentence (3 in this case)

<class 'list'>
['hello. ', 'how are you? ', 'I am fine! ']
hello.  how are you?  I am fine! 


In [8]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature = 0.1)

In [9]:
from llama_index.core import ServiceContext

# 1. Use node parser to break/window the sentences
# 2. Embed the sentences
sentence_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    # embed_model="local:BAAI/bge-large-en-v1.5",
    node_parser= node_parser
)

C:\Users\wongs\AppData\Local\Temp\ipykernel_29224\3019644239.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  sentence_context = ServiceContext.from_defaults(


In [10]:
from llama_index.core import VectorStoreIndex

# 3. Store it into the vector database
sentence_index = VectorStoreIndex.from_documents(
    [document], service_context=sentence_context
)

In [11]:
# This will just save it without indexing it again
sentence_index.storage_context.persist(persist_dir="./sentence_index")

In [12]:
# # This code retrieve the vector database if available already
# import os
# from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
# 
# sentence_index = load_index_from_storage(
#     StorageContext.from_defaults(persist_dir="./sentence_index"),
#     sentence_context = sentence_context
# )

## Postprocessor

- Combine the nodes together according to the window size

In [13]:
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor

postproc = MetadataReplacementPostProcessor(target_metadata_key="window")

In [14]:
from llama_index.core.schema import NodeWithScore
from copy import deepcopy

scored_nodes = [NodeWithScore(node = x, score = 1.0) for x in nodes]
nodes_old = [deepcopy(n) for n in nodes]

In [15]:
nodes_old[1].text

'how are you? '

In [16]:
# This will include window for the node
replaced_nodes = postproc.postprocess_nodes(scored_nodes)
replaced_nodes[1].text

'hello.  how are you?  I am fine! '

## Testing the reranker

In [17]:
# Reranker
from llama_index.core.indices.postprocessor import SentenceTransformerRerank

rerank = SentenceTransformerRerank(
    top_n = 2,
    model = "BAAI/bge-reranker-base"
)

In [18]:
from llama_index.core import QueryBundle
from llama_index.core.schema import TextNode, NodeWithScore

query = QueryBundle("I want a dog.")
scored_nodes = [
    NodeWithScore(node=TextNode(text="This is a cat"), score = 0.6),
    NodeWithScore(node=TextNode(text="This is a dog"), score = 0.4)
]

In [19]:
reranked_nodes = rerank.postprocess_nodes(
    scored_nodes, query_bundle=query
)

In [20]:
print([(x.text, x.score) for x  in reranked_nodes])

[('This is a dog', 0.91827357), ('This is a cat', 0.0014040732)]


#### So reranker will rerank the output of the RAG (in this case, LLM output for other places)

## Real reranker
 - to rerank the window (1,3,5,7), where originally the score is for one node only

In [21]:
# Get the top 6 (k = 6)  most useful query from vector database
# Then, filter the top 2 (n=2) using the reranker

# This is the pipeline
sentence_window_engine = sentence_index.as_query_engine(
    similarity_top_k = 6, node_postprocessors = [postproc, rerank]
)

In [22]:
window_response = sentence_window_engine.query(
    "What are the keys to building a career in AI?"
)

In [23]:
from llama_index.core.response.notebook_utils import display_response

display_response(window_response)
print(window_response)

**`Final Response:`** The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.

The keys to building a career in AI include learning foundational technical skills, working on projects, finding a job, and being part of a supportive community.


In [24]:
from utils import get_sentence_window_query_engine, build_sentence_window_index

index = build_sentence_window_index(
    documents = [document],
    llm = OpenAI(model="gpt-3.5-turbo", temperature= 0.1),
    save_dir="./sentence_index"
)

✅ In Answer relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wongs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
query_engine = get_sentence_window_query_engine(sentence_index = index, similarity_top_k= 6)

## Evaluation

- When context relevance is low, groundedness is low, as the llm will choose to use pretrained knowledge instead of those from RAG

In [26]:
eval_questions = []
with open('./generated_questions.txt', 'r') as file:
    for line in file:
        item = line.strip()
        eval_questions.append(item)


In [27]:
eval_questions

['In the context of job switching, what advantages does working in a startup offer overt a big company, particularly for someone looking to transition into an AI role?',
 'Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?',
 'In the context of project selection and execution, explain the difference between the "Ready, Aim, Fire" and "Ready, Fire, Aim" approaches. Provide examples where each approach might be more benficial.',
 'Disucuss the importance of working on projects that complement your career goals. How can these projects serve as stepping sotnes to larger projects and contribute to technical growth and skill progression?',
 'According to the text ,what are some of the key areas of mathematics that are relevant to machine learning and why are they important in the field of AI?',
 "THe text mentions BG FOgg's approach to building new habits

In [28]:
from trulens_eval import Tru

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            print(question)
            response = query_engine.query(question)
        

In [29]:
tru = Tru()
tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


## Window size = 1

In [30]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm, 
    embed_model = "local:BAAI/bge-small-en-v1.5",
    save_dir="./sentence_index_1",
    sentence_window_size = 1
)

In [31]:
sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index = sentence_index_1
)

In [32]:
import utils
from utils import get_prebuilt_trulens_recorder

tru_recorder_1 = utils.get_prebuilt_trulens_recorder(
    sentence_window_engine=sentence_window_engine_1,
    app_id= "sentence window engine 1"
)

In [33]:
run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder_1, query_engine=sentence_window_engine_1)

In the context of job switching, what advantages does working in a startup offer overt a big company, particularly for someone looking to transition into an AI role?
Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

In the context of project selection and execution, explain the difference between the "Ready, Aim, Fire" and "Ready, Fire, Aim" approaches. Provide examples where each approach might be more benficial.


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

...

In the introduction, the author compares AI coding to a form of literacy. Explain this comparison and discuss how AI coding could be beneficial in everyday professions, using the example of a pizza maker provided in the text.


Groundedness per statement in source:   0%|          | 0/6 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

According to the text, what are the three key steps of career growth in the field of AI? Provide a brief explanation of each step


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In [34]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Dashboard started at http://10.89.63.194:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Sentence window = 3

In [35]:
sentence_index_3 = build_sentence_window_index(
    documents,
    llm, 
    embed_model = "local:BAAI/bge-small-en-v1.5",
    save_dir="./sentence_index_3",
    sentence_window_size = 3
)

sentence_window_engine_3 = get_sentence_window_query_engine(
    sentence_index = sentence_index_3
)

tru_recorder_3 = utils.get_prebuilt_trulens_recorder(
    sentence_window_engine=sentence_window_engine_3,
    app_id= "sentence window engine 3"
)

run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder_3, query_engine=sentence_window_engine_3)

Groundedness per statement in source:   0%|          | 0/6 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In the context of job switching, what advantages does working in a startup offer overt a big company, particularly for someone looking to transition into an AI role?


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/15 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/6 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

According to the text, what are the three key steps of career growth in the field of AI? Provide a brief explanation of each step


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

In [36]:
sentence_index_5 = build_sentence_window_index(
    documents,
    llm, 
    embed_model = "local:BAAI/bge-small-en-v1.5",
    save_dir="./sentence_index_5",
    sentence_window_size = 5
)

sentence_window_engine_5 = get_sentence_window_query_engine(
    sentence_index = sentence_index_5
)

tru_recorder_5 = utils.get_prebuilt_trulens_recorder(
    sentence_window_engine=sentence_window_engine_5,
    app_id= "sentence window engine 5"
)

run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder_5, query_engine=sentence_window_engine_5)

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In the context of job switching, what advantages does working in a startup offer overt a big company, particularly for someone looking to transition into an AI role?


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In the introduction, the author compares AI coding to a form of literacy. Explain this comparison and discuss how AI coding could be beneficial in everyday professions, using the example of a pizza maker provided in the text.


Groundedness per statement in source:   0%|          | 0/7 [00:00<?, ?it/s]

According to the text, what are the three key steps of career growth in the field of AI? Provide a brief explanation of each step


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In [37]:
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://10.89.63.194:8501


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### Discussions
3- just nice, more relevant context is given, context relevance increase
5 - too many irrelevant context is included, groundedness decrease, llm might choose to answer itself instead of referencing to the data

In [43]:
sentence_index_7 = build_sentence_window_index(
    documents,
    llm, 
    embed_model = "local:BAAI/bge-small-en-v1.5",
    save_dir="./sentence_index_7",
    sentence_window_size = 7
)

sentence_window_engine_7 = get_sentence_window_query_engine(
    sentence_index = sentence_index_7
)

tru_recorder_7 = utils.get_prebuilt_trulens_recorder(
    sentence_window_engine=sentence_window_engine_7,
    app_id= "sentence window engine 7"
)

run_evals(eval_questions=eval_questions, tru_recorder=tru_recorder_7, query_engine=sentence_window_engine_7)

In the context of job switching, what advantages does working in a startup offer overt a big company, particularly for someone looking to transition into an AI role?
Explain the scenario planning exercise mentioned in the context of the Covid-19 pandemic. How can this method be applied to personal life situations such as job hunting or exam preparation?


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

In the context of project selection and execution, explain the difference between the "Ready, Aim, Fire" and "Ready, Fire, Aim" approaches. Provide examples where each approach might be more benficial.


Groundedness per statement in source:   0%|          | 0/3 [00:00<?, ?it/s]

...

In the introduction, the author compares AI coding to a form of literacy. Explain this comparison and discuss how AI coding could be beneficial in everyday professions, using the example of a pizza maker provided in the text.


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]

According to the text, what are the three key steps of career growth in the field of AI? Provide a brief explanation of each step


Groundedness per statement in source:   0%|          | 0/4 [00:00<?, ?it/s]